<a href="https://colab.research.google.com/github/faisalisafk/python_scripts/blob/main/Named_entity_custom_with_SPACY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import spacy
from spacy import displacy
from wordcloud import WordCloud, STOPWORDS
from spacy.util import minibatch, compounding
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import re
import random
import math
import copy
from tqdm.notebook import tqdm
random.seed(42)

df = pd.read_csv('data.csv')

In [ ]:
df

,transaction_descriptor,store_number,dataset
0,DOLRTREE 2257 00022574 ROSWELL,2257,train
1,AUTOZONE #3547,3547,train
2,TGI FRIDAYS 1485 0000,1485,train
3,BUFFALO WILD WINGS 003,3,train
4,J. CREW #568 0,568,train
...,...,...,...
295,MCDONALD'S F2151,F2151,test
296,NST BEST BUY #1403 332411,1403,test
297,CVS/PHARMACY #06689,6689,test
298,BANANA REPUBLIC #8109,8109,test


In [ ]:
df.count()

transaction_descriptor    300
store_number              300
dataset                   300
dtype: int64

In [ ]:
nlp0 = spacy.load('en_core_web_sm')

In [ ]:
nlp0.pipe_names

['tagger', 'parser', 'ner']

In [ ]:
ner0 = nlp0.get_pipe('ner')

In [ ]:
def do_it(text,sub):
    start = text.find(sub)
    end = len(sub) + start
    entity = (start,end,'number')
    entities = []
    ent_dict = {}
    entities.append(entity)
    ent_dict['entities'] = entities
    #text = text.replace(" ", "0")
    train_item = (text, ent_dict)
    return train_item

In [ ]:
TRAIN_DATA = []

for _, item in df.iterrows():
    train_item = do_it(item['transaction_descriptor'],item['store_number'])
    TRAIN_DATA.append(train_item)
print(len(TRAIN_DATA))
print(TRAIN_DATA[0])

300
('DOLRTREE 2257 00022574 ROSWELL', {'entities': [(9, 13, 'number')]})


In [ ]:
len(TRAIN_DATA)

300

In [ ]:
n_iter = 100

def train_ner(training_data):
    """Steps
    Create a Blank NLP  model object
    Create and add NER to the NLP model
    Add Labels from your training data
    Train  
    """
    best = math.inf
    TRAIN_DATA = training_data
    nlp = spacy.blank("en")  # create blank Language class
    
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
        
    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    disable_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

    with nlp.disable_pipes(*disable_pipes):
        optimizer = nlp.resume_training()

    nlp.begin_training()
    for itn in tqdm(range(n_iter)):
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA)
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                losses=losses,
                sgd=optimizer
            )
        print("Losses", losses)
        if losses['ner'] < best:
            best = losses['ner']
            best_nlp = copy.deepcopy(nlp)
            print('best changed')
    return best_nlp

In [ ]:
%%time
nlp2 = train_ner(TRAIN_DATA)

Created blank 'en' model


  0%|          | 0/100 [00:00<?, ?it/s]

Losses {'ner': 413.72593373944983}
best changed
Losses {'ner': 79.99541784940998}
best changed
Losses {'ner': 40.466200398595376}
best changed
Losses {'ner': 22.217360074093587}
best changed
Losses {'ner': 13.798156686267308}
best changed
Losses {'ner': 6.092264760897475}
best changed
Losses {'ner': 6.20647799620504}
Losses {'ner': 1.6939517880201784}
best changed
Losses {'ner': 6.197460239751856}
Losses {'ner': 1.80081912472055}
Losses {'ner': 5.005371239402939}
Losses {'ner': 0.016030853594206824}
best changed
Losses {'ner': 4.185066981167309}
Losses {'ner': 8.007515204509058}
Losses {'ner': 5.589633744856146}
Losses {'ner': 0.000705895203342449}
best changed
Losses {'ner': 3.3828386760762283}
Losses {'ner': 4.124840493724477}
Losses {'ner': 5.586503174544551}
Losses {'ner': 3.7113863185397293}
Losses {'ner': 1.4639448377022537}
Losses {'ner': 2.011621814266661}
Losses {'ner': 0.04453579118654503}
Losses {'ner': 3.927583444294793e-05}
best changed
Losses {'ner': 1.9709271147794445}
L

In [ ]:
#try random
temp = df.sample()
print(temp)
print("===============================")
text = temp['transaction_descriptor'].values[0]

docx2 = nlp2(text)

for entity in docx2.ents:
    print(entity)

displacy.render(nlp2(docx2.text),style='ent',jupyter=True)

        transaction_descriptor store_number     dataset
115  NST BEST BUY #380  830230          380  validation
380


In [ ]:
#try 223 cuz issues
temp = df.iloc[[223]]
print(temp)
print("===============================")
text = temp['transaction_descriptor'].values[0]

docx2 = nlp2(text)

for entity in docx2.ents:
    print(entity)

displacy.render(nlp2(docx2.text),style='ent',jupyter=True)

        transaction_descriptor store_number dataset
223  NNT POLO/RL WRENTHA130571        13057    test
WRENTHA130571


In [ ]:
#checking how many were good
count = 0
ok = []
for index, row in df.iterrows():

    docx2 = nlp2(row['transaction_descriptor'])
    try:
    
        if str(docx2.ents[0]).lstrip('0') == str(row['store_number']):
            count+=1
            ok.append(index)   
    except:
        continue
print(count)
print(ok)

264
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 66, 67, 68, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 92, 93, 94, 95, 96, 97, 98, 100, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 114, 115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 132, 133, 135, 136, 137, 138, 139, 141, 142, 143, 145, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 161, 162, 163, 164, 165, 166, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 195, 196, 197, 198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 224, 225, 226, 227, 228, 229, 230, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247

In [ ]:
#checking issues
count = 0
for index, row in df.iterrows():
    if index not in ok:
        docx2 = nlp2(row['transaction_descriptor'])
        try:
            result = str(docx2.ents[0])
        except:
            result = 'EMPTYYYYY'
        result = result.lstrip('0')

        print(str(row['transaction_descriptor']) + ' --- ' + str(row['store_number']) + ' --- ',result)
        count += 1

print(count)
 

EXPRESS#0920 --- 920 ---  EMPTYYYYY
MARATHON PETRO045518 --- 45518 ---  PETRO045518
PAPA JOHN'S 01548.COM --- 1548 ---  1548.COM
NNT LANE BRYANT 455460 --- 5546 ---  455460
CNS EXPRESS#0098   670753 --- 98 ---  EXPRESS#0098
NNT SEARS HOMETOWN 231594 --- 2315 ---  231594
NNT FAMOUS FOOTWEAR572264 --- 572264 ---  FOOTWEAR572264
NNT BURLNGTON STORE171793 --- 171793 ---  STORE171793
NNT FAMOUS FOOTWEAR881706 --- 881706 ---  FOOTWEAR881706
MCDONALD'S F35869MCDONALD'S F35869 --- F35869 ---  F35869MCDONALD
NNT FAMOUS FOOTWEAR470811 --- 470811 ---  FOOTWEAR470811
RACETRAC485   00004853 --- 485 ---  RACETRAC485
NNT BURLNGTON STORE472605 --- 472605 ---  STORE472605
NNT FAMOUSFOOTWEAR#570903 --- 570903 ---  FAMOUSFOOTWEAR#570903
PIER 1 IMPORTS00008060 --- 806 ---  EMPTYYYYY
MURPHY7548ATWALMART      LITHIA SPRINGGA --- 7548 ---  EMPTYYYYY
NNT FAMOUS FOOTWEAR830366 --- 830366 ---  FOOTWEAR830366
NNT BURLNGTON STORE001491 --- 1491 ---  STORE001491
NNT BURLNGTON STORE460391 --- 460391 ---  STORE46039